### Tensorflow 提供了一种学习率指数级衰减的方法:

### decayed_learning_rate = learning_rate * decay_rate ^ (global_step / decay_step)

### 其中learning_rate为初始学习率，decay_rate为衰减系数, decay_step为衰减速度

### 该方法对应的函数为tf.train.exponential_decay(), 其中包含staircase参数，当该参数为True时, global_step/decay_step会被转化为整数，此时一般将decay_step设置为 (训练样本总数/每个批次的样本数)，代表每经过一个完整的训练数据，学习率会有一次衰减；当staircase设置为False的时候，global_step/decay_step为连续的浮点数，此时不同的训练数据将会产生不同的学习率，当学习率减小的时候，样本对于模型的训练结果的影响也就会越小。

In [ ]:
import tensorflow as tf

global_step = tf.Variable(0)

learning_rate = tf.train.exponential_decay(
    learning_rate=0.1,
    global_step=global_step,
    decay_steps=100,
    decay_rate=0.96,
    staircase=True)

train_op = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

### 正则化

In [ ]:
import tensorflow as tf

def get_weight(shape, l):
    var = tf.Variable(tf.random_normal(shape), dtype=tf.float32)
    tf.add_to_collection('losses', tf.contrib.layers.l2_regularizer(l)(var))
    return var


layer_dimension = [2, 10, 3]

for i in range(1, layer_dimension)

### 滑动平均

### tf.train.ExponentialMovingAverage实现了滑动平均模型，其对于每个变量都会维护一个影子变量，影子变量的初始值等于该变量的初始值，当该变量的值更新的时候， 其对应的影子变量也会更新:

### shadow_variable = decay * shadow_variable + (1 - decay) * variable

### 从中可以看出，decay(衰减率)越大，模型越稳定

### 一般在训练的前期，模型的更新速度需要快，所以decay应该较小，训练后期模型的更新速度需要变慢，即decay应该较大

### decay = min {init_decay, (1 + num_updates)/(10 + num_updates)}      (ExponentialMovingAverage利用该式控制衰减率的变化)

In [ ]:
# 定义需要滑动平均的变量与num_updates
import tensorflow as tf
v = tf.Variable(0, dtype=tf.float32)
step = tf.Variable(0, trainable=False)

# 声明滑动平均模型对象
ema = tf.train.ExponentialMovingAverage(0.99, step)
# 定义更新变量的滑动平均操作
maintain_averages_op = ema.apply([v])

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    # 将变量v由0更新到5
    sess.run(tf.assign(v, 5))
    # 获取变量的滑动平均值（也就是其影子变量的值）
    # 首先其衰减率为: min{0.99, (1+step)/(10+step)}=0.1, 初始影子变量为0，更新后的影子变量为: 0.1*0+0.9*5=4.5
    sess.run(maintain_averages_op)
    print(sess.run([v, ema.average(v)]))
    
    # 赋值更新
    sess.run(tf.assign(v, 10))
    sess.run(maintain_averages_op)
    print(sess.run([v, ema.average(v)]))